In [54]:
# Depedencies required for dataset import
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import os, json, math, random, time
import random
from dotenv import load_dotenv
from collections import defaultdict
import qci_client as qc
from pprint import pprint

In [55]:
dataset = pd.read_csv("./supply_chain_data.csv")

# Data Description


In [56]:
dataset.shape  # dimension

(100, 24)

In [57]:
dataset.dtypes  # variables and types

Product type                object
SKU                         object
Price                      float64
Availability                 int64
Number of products sold      int64
Revenue generated          float64
Customer demographics       object
Stock levels                 int64
Lead times                   int64
Order quantities             int64
Shipping times               int64
Shipping carriers           object
Shipping costs             float64
Supplier name               object
Location                    object
Lead time                    int64
Production volumes           int64
Manufacturing lead time      int64
Manufacturing costs        float64
Inspection results          object
Defect rates               float64
Transportation modes        object
Routes                      object
Costs                      float64
dtype: object

In [58]:
dataset.info(memory_usage=True, show_counts=True)  # column structure types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product type             100 non-null    object 
 1   SKU                      100 non-null    object 
 2   Price                    100 non-null    float64
 3   Availability             100 non-null    int64  
 4   Number of products sold  100 non-null    int64  
 5   Revenue generated        100 non-null    float64
 6   Customer demographics    100 non-null    object 
 7   Stock levels             100 non-null    int64  
 8   Lead times               100 non-null    int64  
 9   Order quantities         100 non-null    int64  
 10  Shipping times           100 non-null    int64  
 11  Shipping carriers        100 non-null    object 
 12  Shipping costs           100 non-null    float64
 13  Supplier name            100 non-null    object 
 14  Location                 10

In [59]:
dataset.describe()  # statistical description

,Price,Availability,Number of products sold,Revenue generated,Stock levels,Lead times,Order quantities,Shipping times,Shipping costs,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Defect rates,Costs
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000
mean,49.462461,48.400000,460.990000,5776.048187,47.770000,15.960000,49.220000,5.750000,5.548149,17.080000,567.840000,14.77000,47.266693,2.277158,529.245782
std,31.168193,30.743317,303.780074,2732.841744,31.369372,8.785801,26.784429,2.724283,2.651376,8.846251,263.046861,8.91243,28.982841,1.461366,258.301696
min,1.699976,1.000000,8.000000,1061.618523,0.000000,1.000000,1.000000,1.000000,1.013487,1.000000,104.000000,1.00000,1.085069,0.018608,103.916248
25%,19.597823,22.750000,184.250000,2812.847151,16.750000,8.000000,26.000000,3.750000,3.540248,10.000000,352.000000,7.00000,22.983299,1.009650,318.778455
50%,51.239831,43.500000,392.500000,6006.352023,47.500000,17.000000,52.000000,6.000000,5.320534,18.000000,568.500000,14.00000,45.905622,2.141863,520.430444
75%,77.198228,75.000000,704.250000,8253.976921,73.000000,24.000000,71.250000,8.000000,7.601695,25.000000,797.000000,23.00000,68.621026,3.563995,763.078231
max,99.171329,100.000000,996.000000,9866.465458,100.000000,30.000000,96.000000,10.000000,9.929816,30.000000,985.000000,30.00000,99.466109,4.939255,997.413450


In [60]:
dataset.head(10)  # top 10 rows

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632
5,haircare,SKU5,1.699976,87,147,2828.348746,Non-binary,90,27,66,...,Bangalore,10,104,17,56.766476,Fail,2.779194,Road,Route A,235.461237
6,skincare,SKU6,4.078333,48,65,7823.476560,Male,11,15,58,...,Kolkata,14,314,24,1.085069,Pending,1.000911,Sea,Route A,134.369097
7,cosmetics,SKU7,42.958384,59,426,8496.103813,Female,93,17,11,...,Bangalore,22,564,1,99.466109,Fail,0.398177,Road,Route C,802.056312
8,cosmetics,SKU8,68.717597,78,150,7517.363211,Female,5,10,15,...,Mumbai,13,769,8,11.423027,Pending,2.709863,Sea,Route B,505.557134
9,skincare,SKU9,64.015733,35,980,4971.145988,Unknown,14,27,83,...,Chennai,29,963,23,47.957602,Pending,3.844614,Rail,Route B,995.929461


# Data Cleaning


## NA Values


In [61]:
np.sum(dataset.isna())  # check for na values

c:\Users\tamka\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


Product type               0
SKU                        0
Price                      0
Availability               0
Number of products sold    0
Revenue generated          0
Customer demographics      0
Stock levels               0
Lead times                 0
Order quantities           0
Shipping times             0
Shipping carriers          0
Shipping costs             0
Supplier name              0
Location                   0
Lead time                  0
Production volumes         0
Manufacturing lead time    0
Manufacturing costs        0
Inspection results         0
Defect rates               0
Transportation modes       0
Routes                     0
Costs                      0
dtype: int64

## Duplicates


In [62]:
dataset.duplicated().sum()  # check for dupes

0

## Conversion of column names to snake case


In [63]:
dataset.columns = [col.lower().replace(" ", "_") for col in dataset.columns]
dataset.columns

Index(['product_type', 'sku', 'price', 'availability',
       'number_of_products_sold', 'revenue_generated', 'customer_demographics',
       'stock_levels', 'lead_times', 'order_quantities', 'shipping_times',
       'shipping_carriers', 'shipping_costs', 'supplier_name', 'location',
       'lead_time', 'production_volumes', 'manufacturing_lead_time',
       'manufacturing_costs', 'inspection_results', 'defect_rates',
       'transportation_modes', 'routes', 'costs'],
      dtype='object')

# Synthetic Data Generation

Since 100 rows might not be sufficient for our project to showcase a difference in performance, we'll generate 100k rows of synthetic data based on the original dataset.

The numeric columns will be based on the original mean and a standard deviation of 3.
The string columns will follow the original values given in the dataset.


## Configurations


In [64]:
NUM_SYNTHETIC_ROWS = 100000
SEED = 42
MEAN_SCALE = 1
MEAN_OFFSET = 0
STANDARD_DEVIATION = 3

np.random.seed(SEED)

## Separation of Numeric and String Columns


In [65]:
numeric_cols = dataset.select_dtypes(include=[np.number]).columns
string_cols = dataset.select_dtypes(include=["object", "string", "category"]).columns

print("Numeric columns (", numeric_cols.size, "):", list(numeric_cols))
print("String columns (", string_cols.size, "):", list(string_cols))

Numeric columns ( 15 ): ['price', 'availability', 'number_of_products_sold', 'revenue_generated', 'stock_levels', 'lead_times', 'order_quantities', 'shipping_times', 'shipping_costs', 'lead_time', 'production_volumes', 'manufacturing_lead_time', 'manufacturing_costs', 'defect_rates', 'costs']
String columns ( 9 ): ['product_type', 'sku', 'customer_demographics', 'shipping_carriers', 'supplier_name', 'location', 'inspection_results', 'transportation_modes', 'routes']


## Generation of Synthetic Data for Numeric Columns


In [66]:
synthetic_numeric = pd.DataFrame()

for col in numeric_cols:
    mean_val = dataset[col].mean() * MEAN_SCALE + MEAN_OFFSET
    synthetic_numeric[col] = np.random.normal(
        loc=mean_val, scale=STANDARD_DEVIATION, size=NUM_SYNTHETIC_ROWS
    )

    if np.issubdtype(dataset[col].dtype, np.integer):
        # Ensure that the original int columns stays int
        synthetic_numeric[col] = synthetic_numeric[col].round().astype(int)

print("Synthetic Numeric Data", synthetic_numeric.shape)
synthetic_numeric.tail(10)

Synthetic Numeric Data (100000, 15)


,price,availability,number_of_products_sold,revenue_generated,stock_levels,lead_times,order_quantities,shipping_times,shipping_costs,lead_time,production_volumes,manufacturing_lead_time,manufacturing_costs,defect_rates,costs
99990,51.394205,44,461,5776.509233,47,13,49,6,5.983505,17,567,17,43.542572,-1.068146,527.627460
99991,51.940238,50,455,5776.740630,50,14,49,3,7.395612,24,566,12,46.329913,4.144333,529.953724
99992,48.822352,46,460,5776.588047,44,18,51,4,2.888499,22,565,15,48.738104,2.847237,526.899451
99993,50.105086,51,463,5779.173835,46,18,49,12,7.415677,20,563,13,44.919218,8.149500,530.018552
99994,53.408729,47,465,5774.493725,49,13,56,8,3.452690,17,567,10,44.781101,3.720170,527.136398
99995,48.786787,45,460,5774.384820,52,16,49,3,6.048002,18,567,17,48.107720,6.876574,534.997305
99996,47.753129,50,459,5769.099043,47,14,48,11,9.546258,17,564,10,41.967264,4.297456,526.895081
99997,50.690017,49,459,5778.542855,47,16,48,9,5.758243,17,567,22,44.543516,1.013788,527.049298
99998,48.829186,51,458,5777.435779,45,16,51,6,8.590805,21,567,13,47.261134,0.374709,528.058140
99999,49.822650,45,460,5775.636116,43,16,51,9,7.203290,19,575,15,40.502722,-0.627771,534.747633


## Generation of Synthetic Data for String Columns


In [67]:
synthetic_strings = pd.DataFrame()

for col in string_cols:
    # I'm not gonna use .unique() here, since it might be better to keep
    # the original spread/ratio of the dupe values
    synthetic_strings[col] = np.random.choice(
        dataset[col].dropna(), size=NUM_SYNTHETIC_ROWS
    )

print("Synthetic String Data", synthetic_strings.shape)
synthetic_strings.tail(10)

Synthetic String Data (100000, 9)


,product_type,sku,customer_demographics,shipping_carriers,supplier_name,location,inspection_results,transportation_modes,routes
99990,cosmetics,SKU75,Male,Carrier B,Supplier 1,Kolkata,Pass,Rail,Route A
99991,haircare,SKU20,Male,Carrier C,Supplier 5,Kolkata,Pending,Rail,Route B
99992,skincare,SKU92,Unknown,Carrier C,Supplier 1,Chennai,Fail,Sea,Route B
99993,haircare,SKU85,Male,Carrier C,Supplier 2,Kolkata,Fail,Rail,Route C
99994,haircare,SKU47,Female,Carrier C,Supplier 1,Mumbai,Pass,Sea,Route A
99995,haircare,SKU5,Female,Carrier A,Supplier 1,Kolkata,Pass,Rail,Route A
99996,skincare,SKU4,Non-binary,Carrier B,Supplier 2,Bangalore,Pass,Sea,Route A
99997,skincare,SKU54,Female,Carrier A,Supplier 5,Mumbai,Pending,Rail,Route A
99998,haircare,SKU72,Unknown,Carrier C,Supplier 3,Delhi,Fail,Air,Route A
99999,skincare,SKU10,Male,Carrier C,Supplier 2,Bangalore,Pass,Road,Route C


## Combining Numeric and String Data


In [68]:
dataset_synthetic = pd.concat([synthetic_numeric, synthetic_strings], axis=1)

dataset_synthetic = dataset_synthetic[dataset.columns]

dataset_synthetic.reset_index(drop=True, inplace=True)

print("Original dataset shape:", dataset.shape)
print("Synthetic dataset shape:", dataset_synthetic.shape)

dataset_synthetic.head()

Original dataset shape: (100, 24)
Synthetic dataset shape: (100000, 24)


,product_type,sku,price,availability,number_of_products_sold,revenue_generated,customer_demographics,stock_levels,lead_times,order_quantities,...,location,lead_time,production_volumes,manufacturing_lead_time,manufacturing_costs,inspection_results,defect_rates,transportation_modes,routes,costs
0,haircare,SKU14,50.952604,51,466,5771.848722,Unknown,57,13,55,...,Chennai,25,568,15,47.413995,Pending,2.466346,Rail,Route B,528.063779
1,haircare,SKU9,49.047668,45,461,5777.691656,Unknown,47,13,49,...,Kolkata,18,567,13,53.268131,Pending,7.514651,Sea,Route A,528.965788
2,skincare,SKU50,51.405527,50,457,5775.685900,Male,43,18,47,...,Bangalore,12,571,19,48.049698,Pass,4.395606,Air,Route B,526.226741
3,cosmetics,SKU59,54.031551,47,457,5775.198396,Unknown,49,17,48,...,Mumbai,17,568,15,42.219199,Pass,1.656860,Rail,Route A,527.782457
4,haircare,SKU35,48.760001,47,460,5778.350895,Male,48,17,54,...,Bangalore,19,570,10,48.875692,Pending,-1.685770,Sea,Route A,528.764678


# Gurobi (Classical) Experiment

Given that Gurobi do not directly handle higher order terms in the objective function polynomial (HUBO), we introduce auxiliary variables and quadratic interactions with those auxiliary variables:

Replace a product $a = xy$ of binary variables, $x_i \in \{0,1\}$, with a new binary auxiliary $a$, then add the QUBO penalty:
$$
    P_M(a,x,y)\;=\;M\big(xy - 2ax - 2ay + 3a\big).
$$
where:
- $P_M=0$ (Penalty Term) `iff` $a = xy$; otherwise $P_M \geq 1$ (the order-reduction/quadratic-penalty gadget $xy - 2ax - 2ay + 3a$ encodes the penalty logic)
- Pick $M$ (Penalty Weight) "large enough" (rule-of-thumb should be larger than coefficients within the order-reduction gadget)


`Reference:` "Classical Combinatorial Optimization Scaling for Randomy Ising Models on 2D Heavy-Hex Graphs" by E Pelofske, etc.

In [69]:
# === global knobs you change in ONE place ===
REPLICAS = 1       # same value you'd put in Gurobi
JITTERS   = 0.0        # same jitter for both backends

# Conservative per-job variable cap (adjust up if you know your org limit)
DEVICE_VAR_CAP = 135
# ============================================

In [70]:
# === Pure QUBO for 3rd-order HUBO with pick-one constraints, using ALL rows ===
# - Every row contributes directly (no fancy wrangling)
# - Exactly-one via quadratic penalties
# - Cubic term x_i y_j z_k -> quadratic using aux t_{jk} = y_j * z_k (AND gadget)
# - Gurobi: NonConvex=2


# -----------------------------
# 0) CONFIG: column mapping
# -----------------------------
COL_I = "supplier_name"         # i in I
COL_J = "product_type"          # j in J
COL_K = "shipping_carriers"     # k in K

COL_COST_SUP = "costs"                     # for a_i
COL_COST_MAN = "manufacturing_costs"       # for b_j
COL_COST_CAR = "shipping_costs"            # for c_k (fallback to costs if missing)

COL_LEAD = "lead_time"         # or 'lead_times'
COL_DEF  = "defect_rates"

# Feature->$ weights (simple, no normalization)
W_LT  = 1.0
W_DEF = 50.0

# Optional solver controls
TIME_LIMIT = None   # e.g., 60
MIP_GAP    = None   # e.g., 0.01


In [71]:
# -----------------------------
# 1) Sets & indices
# -----------------------------
def build_sets(df: pd.DataFrame):
    I_vals = df[COL_I].astype(str).dropna().unique().tolist()
    J_vals = df[COL_J].astype(str).dropna().unique().tolist()
    K_vals = df[COL_K].astype(str).dropna().unique().tolist()
    I_vals.sort(); J_vals.sort(); K_vals.sort()

    var_idx = {}
    next_idx = 1
    I_idx, J_idx, K_idx = [], [], []

    for i in I_vals:
        var_idx[('I', i)] = next_idx; I_idx.append(next_idx); next_idx += 1
    for j in J_vals:
        var_idx[('J', j)] = next_idx; J_idx.append(next_idx); next_idx += 1
    for k in K_vals:
        var_idx[('K', k)] = next_idx; K_idx.append(next_idx); next_idx += 1

    base_n = next_idx - 1
    return I_vals, J_vals, K_vals, var_idx, I_idx, J_idx, K_idx, base_n

In [72]:
# -----------------------------
# 2) Build HUBO coefficients (ALL rows)
#     H = Σ a_i x_i + Σ b_j y_j + Σ c_k z_k
#       - Σ s_ij x_i y_j - Σ u_ik x_i z_k - Σ v_jk y_j z_k
#       - Σ w_ijk x_i y_j z_k
# -----------------------------
def build_hubo(df: pd.DataFrame, I_vals, J_vals, K_vals, var_idx):
    a = defaultdict(float); b = defaultdict(float); c = defaultdict(float)
    quad = {}   # {(i,j): coeff}
    cubic = {}  # {(i,j,k): coeff}

    use_car_cost = (COL_COST_CAR in df.columns)

    # ---- GLOBAL MEANS (baselines) ----
    g_lt   = float(df[COL_LEAD].mean())
    g_dr   = float(df[COL_DEF].mean())
    g_mc   = float(df[COL_COST_MAN].mean())
    g_cost = float(df[COL_COST_SUP].mean())
    g_car  = float(df[COL_COST_CAR].mean()) if use_car_cost else g_cost

    # ---- GROUP MEANS (not sums) ----
    # Bases
    mean_cost_I = df.groupby(COL_I)[COL_COST_SUP].mean().to_dict()
    mean_mc_J   = df.groupby(COL_J)[COL_COST_MAN].mean().to_dict()
    mean_cost_K = (df.groupby(COL_K)[COL_COST_CAR].mean().to_dict()
                   if use_car_cost else df.groupby(COL_K)[COL_COST_SUP].mean().to_dict())

    # Pairs (lead time, defects)
    lt_IJ = df.groupby([COL_I, COL_J])[COL_LEAD].mean()
    dr_IJ = df.groupby([COL_I, COL_J])[COL_DEF].mean()
    lt_IK = df.groupby([COL_I, COL_K])[COL_LEAD].mean()
    dr_IK = df.groupby([COL_I, COL_K])[COL_DEF].mean()
    lt_JK = df.groupby([COL_J, COL_K])[COL_LEAD].mean()
    dr_JK = df.groupby([COL_J, COL_K])[COL_DEF].mean()

    # Triples
    lt_IJK = df.groupby([COL_I, COL_J, COL_K])[COL_LEAD].mean()
    dr_IJK = df.groupby([COL_I, COL_J, COL_K])[COL_DEF].mean()
    mc_IJK = df.groupby([COL_I, COL_J, COL_K])[COL_COST_MAN].mean()

    # ---- BASE terms (means) ----
    for i in I_vals:
        a[var_idx[('I', i)]] = mean_cost_I.get(i, g_cost)
    for j in J_vals:
        b[var_idx[('J', j)]] = mean_mc_J.get(j, g_mc)
    for k in K_vals:
        c[var_idx[('K', k)]] = mean_cost_K.get(k, g_car)

    # ---- PAIR "savings" as improvements vs global ----
    def add_q(d, i, j, wgt):
        key = (i, j) if i < j else (j, i)
        d[key] = d.get(key, 0.0) + wgt

    for i in I_vals:
        for j in J_vals:
            lt = float(lt_IJ.get((i, j), g_lt)); dr = float(dr_IJ.get((i, j), g_dr))
            saving = -(W_LT*(lt - g_lt) + W_DEF*(dr - g_dr))
            add_q(quad, var_idx[('I', i)], var_idx[('J', j)], saving)

    for i in I_vals:
        for k in K_vals:
            lt = float(lt_IK.get((i, k), g_lt)); dr = float(dr_IK.get((i, k), g_dr))
            saving = -(W_LT*(lt - g_lt) + W_DEF*(dr - g_dr))
            add_q(quad, var_idx[('I', i)], var_idx[('K', k)], saving)

    for j in J_vals:
        for k in K_vals:
            lt = float(lt_JK.get((j, k), g_lt)); dr = float(dr_JK.get((j, k), g_dr))
            saving = -(W_LT*(lt - g_lt) + W_DEF*(dr - g_dr))
            add_q(quad, var_idx[('J', j)], var_idx[('K', k)], saving)

    # ---- TRIPLE "savings" vs global ----
    for i in I_vals:
        for j in J_vals:
            for k in K_vals:
                lt = float(lt_IJK.get((i, j, k), g_lt))
                dr = float(dr_IJK.get((i, j, k), g_dr))
                mc = float(mc_IJK.get((i, j, k), g_mc))
                saving = -(0.8*W_LT*(lt - g_lt) + 0.2*W_DEF*(dr - g_dr) + 0.5*(mc - g_mc))
                ii = var_idx[('I', i)]; jj = var_idx[('J', j)]; kk = var_idx[('K', k)]
                cubic[(ii, jj, kk)] = cubic.get((ii, jj, kk), 0.0) + saving

    # merge bases to linear
    lin = {}
    lin.update(a); lin.update(b); lin.update(c)
    return lin, quad, cubic

def rescale_hubo(lin_h, quad_h, cubic_h, target_max=1e4):
    max_abs = 1.0
    if lin_h:   max_abs = max(max_abs, max(abs(v) for v in lin_h.values()))
    if quad_h:  max_abs = max(max_abs, max(abs(v) for v in quad_h.values()))
    if cubic_h: max_abs = max(max_abs, max(abs(v) for v in cubic_h.values()))
    scale = max_abs / target_max
    if scale <= 1.0:
        return lin_h, quad_h, cubic_h, 1.0
    lin_s   = {i: v/scale for i, v in lin_h.items()}
    quad_s  = {(i,j): v/scale for (i,j), v in quad_h.items()}
    cubic_s = {(i,j,k): v/scale for (i,j,k), v in cubic_h.items()}
    return lin_s, quad_s, cubic_s, scale



In [73]:
# -----------------------------
# 3) HUBO -> QUBO with t_{jk} = y_j z_k
# -----------------------------
def hubo_to_qubo_with_yz_aux(lin_h, quad_h, cubic_h, base_n, nI, nJ, nK, P_AND):
    lin_q  = dict(lin_h)
    quad_q = dict(quad_h)
    next_idx = base_n + 1

    def add_q(i, j, w):
        a_, b_ = (i, j) if i < j else (j, i)
        quad_q[(a_, b_)] = quad_q.get((a_, b_), 0.0) + w

    def add_l(i, w): lin_q[i] = lin_q.get(i, 0.0) + w

    def enforce_AND(a, b, y, P):
        # P*(ab - 2ay - 2by + 3y)
        add_q(a, b, +P)
        add_q(a, y, -2.0*P)
        add_q(b, y, -2.0*P)
        add_l(y,  3.0*P)

    yz_aux = {}  # (j,k) -> t index

    for (i, j, k), coeff in cubic_h.items():
        key = (j, k)   # ALWAYS y_j z_k
        if key in yz_aux:
            t = yz_aux[key]
        else:
            t = next_idx; next_idx += 1
            yz_aux[key] = t
            enforce_AND(j, k, t, P_AND)  # t = y_j * z_k
        add_q(i, t, coeff)                # coeff * x_i * t_{jk}

    return lin_q, quad_q, next_idx - 1, yz_aux


In [74]:
# -----------------------------
# 4) Pick-one penalties: P * (sum x - 1)^2
# -----------------------------
def add_one_hot_penalty(lin_q, quad_q, idx_list, P):
    for i in idx_list:
        lin_q[i] = lin_q.get(i, 0.0) - P
    n = len(idx_list)
    for a in range(n):
        for b in range(a+1, n):
            i, j = idx_list[a], idx_list[b]
            key = (i, j) if i < j else (j, i)
            quad_q[key] = quad_q.get(key, 0.0) + 2.0*P

In [75]:
# -----------------------------
# 5) Auto-penalty selection (dominates native coeffs)
# -----------------------------
def compute_penalties(lin_h, quad_h, cubic_h, I_idx, J_idx, K_idx, safety=20.0):
    deg = defaultdict(float)
    for i, w in lin_h.items(): deg[i] += abs(w)
    for (i, j), w in quad_h.items():
        a = abs(w); deg[i] += a; deg[j] += a
    for (i, j, k), w in cubic_h.items():
        a = abs(w); deg[i] += a; deg[j] += a; deg[k] += a

    max_I = max((deg[i] for i in I_idx), default=0.0)
    max_J = max((deg[j] for j in J_idx), default=0.0)
    max_K = max((deg[k] for k in K_idx), default=0.0)

    base = max(max_I, max_J, max_K, 1.0)
    P_onehot = safety * base       # strong
    P_and    = 2.0 * P_onehot      # stronger
    return P_onehot, P_and

In [76]:
# -----------------------------
# 6) Solve & report
# -----------------------------
def solve_qubo(lin_q, quad_q, num_vars, names_by_idx, I_idx, J_idx, K_idx):
    m = gp.Model("QUBO_bigdata_pick_one")
    m.Params.NonConvex = 2
    if TIME_LIMIT is not None: m.Params.TimeLimit = TIME_LIMIT
    if MIP_GAP is not None:    m.Params.MIPGap    = MIP_GAP

    x = m.addVars(range(1, num_vars+1), vtype=GRB.BINARY, name="x")

    obj = gp.QuadExpr(0.0)
    for i, w in lin_q.items():
        if 1 <= i <= num_vars:
            obj += w * x[i]
    for (i, j), w in quad_q.items():
        if 1 <= i <= num_vars and 1 <= j <= num_vars:
            obj += w * (x[i]*x[j])

    m.setObjective(obj, GRB.MINIMIZE)
    m.optimize()

    if m.Status != GRB.OPTIMAL:
        print("Status:", m.Status)
        return None, None, m

    sol = {i: int(round(x[i].X)) for i in x.keys()}
    chosen = [names_by_idx[i] for i, v in sol.items() if v == 1 and i in names_by_idx]

    # sanity: exactly one in each group
    countI = sum(sol.get(i,0) for i in I_idx)
    countJ = sum(sol.get(j,0) for j in J_idx)
    countK = sum(sol.get(k,0) for k in K_idx)
    print(f"one-hot counts -> I:{countI} J:{countJ} K:{countK}")
    if (countI, countJ, countK) != (1,1,1):
        print("WARNING: one-hot not satisfied; increase penalties or inspect coefficients.")

    return sol, chosen, m


In [77]:
# -----------------------------
# 7) End-to-end runner
# -----------------------------
def run_qubo(df: pd.DataFrame):
    I_vals, J_vals, K_vals, var_idx, I_idx, J_idx, K_idx, base_n = build_sets(df)

    # readable names
    names_by_idx = {idx: f"I:{key}" for (g,key), idx in var_idx.items() if g=='I'}
    names_by_idx.update({idx: f"J:{key}" for (g,key), idx in var_idx.items() if g=='J'})
    names_by_idx.update({idx: f"K:{key}" for (g,key), idx in var_idx.items() if g=='K'})

    lin_h, quad_h, cubic_h = build_hubo(df, I_vals, J_vals, K_vals, var_idx)
    lin_h, quad_h, cubic_h, scale_used = rescale_hubo(lin_h, quad_h, cubic_h, target_max=1e4)
    print(f"Rescaled native coeffs by 1/{scale_used:.3e}")

    # penalties from data scale
    P_ONEHOT, P_AND = compute_penalties(lin_h, quad_h, cubic_h, I_idx, J_idx, K_idx, safety=8.0)
    print(f"Auto penalties (scaled): P_ONEHOT={P_ONEHOT:.2e}, P_AND={P_AND:.2e}")


    # HUBO -> QUBO with t_{jk}=y_j z_k
    lin_q, quad_q, last_idx, aux_map = hubo_to_qubo_with_yz_aux(
        lin_h, quad_h, cubic_h,
        base_n=base_n, nI=len(I_vals), nJ=len(J_vals), nK=len(K_vals),
        P_AND=P_AND
    )

    # exactly-one penalties
    add_one_hot_penalty(lin_q, quad_q, I_idx, P_ONEHOT)
    add_one_hot_penalty(lin_q, quad_q, J_idx, P_ONEHOT)
    add_one_hot_penalty(lin_q, quad_q, K_idx, P_ONEHOT)

        # ==== replicas benchmark ====
    R = REPLICAS   # set to 10, 25, 50, 100... to scale problem size
    JITTER = JITTERS  # e.g., 1e-3 to break ties across blocks (optional)

    lin_R, quad_R, num_vars_R = replicate_qubo(lin_q, quad_q, last_idx, R=R, jitter=JITTER)
    print(f"[replicas] R={R} -> variables={num_vars_R}, linear terms={len(lin_R)}, quadratic terms={len(quad_R)}")

    model_R, solve_seconds, status = solve_and_time(lin_R, quad_R, num_vars_R, output_flag=1)
    print(f"[replicas] status={status}, time={solve_seconds:.3f}s, objective={model_R.ObjVal:.6f}")


    print(f"|I|={len(I_vals)}, |J|={len(J_vals)}, |K|={len(K_vals)}")
    print(f"Vars base={base_n}, aux={last_idx-base_n}, total={last_idx}")
    print(f"Linear terms={len(lin_q)}, Quadratic terms={len(quad_q)}")

    sol, chosen, model = solve_qubo(lin_q, quad_q, last_idx, names_by_idx, I_idx, J_idx, K_idx)
    if sol is None:
        return

    print("\n== Optimal pick-one-of-each ==")
    I_pick = [n for n in chosen if n.startswith("I:")]
    J_pick = [n for n in chosen if n.startswith("J:")]
    K_pick = [n for n in chosen if n.startswith("K:")]
    print("Supplier:", I_pick)
    print("Manufacturer:", J_pick)
    print("Carrier:", K_pick)
    print("Objective:", model.ObjVal)

In [78]:
# Extra function in-case we want to increase problem size via replicas

def replicate_qubo(lin_q, quad_q, num_vars, R: int, jitter: float = 0.0):
    """
    Make R independent copies (blocks) of a given QUBO.
    - lin_q: {i: w}
    - quad_q: {(i,j): w} with i<j
    - num_vars: number of variables in the base (1..num_vars)
    - R: number of replicas (>=1)
    - jitter: add small random +/- jitter * |w| to each coeff (break symmetry; 0 to disable)
    Returns (lin_R, quad_R, num_vars_R)
    """
    lin_R, quad_R = {}, {}
    for r in range(R):
        off = r * num_vars
        # linear
        for i, w in lin_q.items():
            w2 = w
            if jitter:
                w2 *= (1.0 + jitter * (2*random.random()-1))
            lin_R[off + i] = lin_R.get(off + i, 0.0) + w2
        # quadratic
        for (i, j), w in quad_q.items():
            w2 = w
            if jitter:
                w2 *= (1.0 + jitter * (2*random.random()-1))
            a, b = off + i, off + j
            key = (a, b) if a < b else (b, a)
            quad_R[key] = quad_R.get(key, 0.0) + w2
    return lin_R, quad_R, R * num_vars

def solve_and_time(lin_q, quad_q, num_vars, output_flag=1):
    m = gp.Model("QUBO_replicas")
    m.Params.NonConvex = 2
    m.Params.OutputFlag = output_flag
    # Optional: set your TimeLimit / MIPGap here if you want
    x = m.addVars(range(1, num_vars+1), vtype=GRB.BINARY, name="x")
    obj = gp.QuadExpr()
    for i, w in lin_q.items():
        if 1 <= i <= num_vars: obj += w * x[i]
    for (i, j), w in quad_q.items():
        if 1 <= i <= num_vars and 1 <= j <= num_vars:
            obj += w * (x[i] * x[j])
    m.setObjective(obj, GRB.MINIMIZE)
    t0 = time.perf_counter()
    m.optimize()
    t1 = time.perf_counter()
    status = m.Status
    return m, (t1 - t0), status


Run the cell below to run Gurobi

In [79]:
start = time.perf_counter()   # Start timing
run_qubo(dataset_synthetic)
total = sum(i**2 for i in range(100000))
end = time.perf_counter()     # End timing
elapsed = end - start
print(f"Elapsed time: {elapsed:.6f} seconds")

Rescaled native coeffs by 1/1.000e+00
Auto penalties (scaled): P_ONEHOT=4.37e+03, P_AND=8.74e+03
[replicas] R=1 -> variables=20, linear terms=20, quadratic terms=118
Set parameter NonConvex to value 2
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Non-default parameters:
NonConvex  2

Optimize a model with 0 rows, 20 columns and 0 nonzeros
Model fingerprint: 0x8346902e
Model has 118 quadratic objective terms
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [4e+03, 3e+04]
  QObjective range [4e-02, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000
Found heuristic solution: objective -12528.18723
Found heuri

# Quantum (DIRAC-3) Experiment

- Native HUBO: DIRAC-3 handles higher-order terms directly, so we don’t add auxiliaries or AND-gadgets. We submit the original polynomial:

$$
\min_x\; \sum_i h_i x_i + \sum_{i<j} J_{ij} x_ix_j + \sum_{i<j<k} K_{ijk} x_ix_jx_k.
$$

- Constraints:
    - Global picks: use `sum_constraint = 3` to force exactly three 1’s overall.
    - Per-group “pick-one”: add light quadratic penalties $\lambda(\sum_{g} x_g - 1)^2$ per group to steer the 3 ones into (1 supplier, 1 product, 1 carrier) without huge $M$.

- Scaling: After adding penalties, rescale coefficients so $\max|\,\text{coeff}\,|$ is in a stable range (e.g., $10^3\!\sim\!10^4$) for better sampling.

- Readout: Samples may be spins (−1/+1) or soft [0,1]. Decode via $b=(s+1)/2$ or threshold at 0.5; if a group is empty after thresholding, pick the highest raw entry in that group.

- Fairness vs. classical: Same objective; classical needs QUBO reduction and big $M$, while quantum uses native HUBO + global cardinality + light group penalties. Compare by energy, feasibility (1-per-group), and time.

In [80]:
# ==============================
# DIRAC-3: native HUBO version
# (no AND gadget; keep cubic)
# ==============================

# === global knobs you change in ONE place ===
REPLICAS = 1        # same intent as your Gurobi replicas block
JITTERS  = 0.0      # tiny random wiggle; set 0.0 to keep things exact
DEVICE_VAR_CAP = 135  # rough ceiling to avoid too many variables per job
# ============================================

# If you have these helpers in your environment, great.
# If not, the code still writes a self-contained HUBO polynomial file.
try:
    from your_dirac3_helpers import run_dirac3_native_hubo, run_dirac3_tiled, normalize_polynomial_inplace
except Exception:
    run_dirac3_native_hubo = None
    run_dirac3_tiled = None
    normalize_polynomial_inplace = None

# -----------------------------
# 0) CONFIG: column mapping
# -----------------------------
COL_I = "supplier_name"         # i in I
COL_J = "product_type"          # j in J
COL_K = "shipping_carriers"     # k in K

COL_COST_SUP = "costs"               # supplier cost a_i
COL_COST_MAN = "manufacturing_costs" # manufacturer cost b_j
COL_COST_CAR = "shipping_costs"      # carrier cost c_k (fallback to costs if missing)

COL_LEAD = "lead_time"
COL_DEF  = "defect_rates"

# Feature weights. Think of these as how “loudly” each thing matters.
W_LT  = 1.0   # lead time matters, but gently
W_DEF = 50.0  # defects matter a LOT (we really hate broken stuff)

# Optional solver controls (only used if you also run the fallback Gurobi path elsewhere)
TIME_LIMIT = None
MIP_GAP    = None


In [81]:
# -----------------------------
# 1) Build index sets (turn names -> integer ids)
# -----------------------------
def build_sets(df: pd.DataFrame):
    """
    We turn text labels like 'Acme Co' into integer ids 1..N.
    Computers like numbers; solvers like numbers even more.
    """
    I_vals = df[COL_I].astype(str).dropna().unique().tolist()
    J_vals = df[COL_J].astype(str).dropna().unique().tolist()
    K_vals = df[COL_K].astype(str).dropna().unique().tolist()
    I_vals.sort(); J_vals.sort(); K_vals.sort()

    var_idx = {}
    next_idx = 1
    I_idx, J_idx, K_idx = [], [], []

    for i in I_vals:
        var_idx[('I', i)] = next_idx; I_idx.append(next_idx); next_idx += 1
    for j in J_vals:
        var_idx[('J', j)] = next_idx; J_idx.append(next_idx); next_idx += 1
    for k in K_vals:
        var_idx[('K', k)] = next_idx; K_idx.append(next_idx); next_idx += 1

    base_n = next_idx - 1
    return I_vals, J_vals, K_vals, var_idx, I_idx, J_idx, K_idx, base_n


In [82]:
# -----------------------------
# 2) Build HUBO coefficients (ALL rows)
# H(x,y,z) =
#   Σ a_i x_i  +  Σ b_j y_j  +  Σ c_k z_k
# + Σ q_ij x_i y_j + Σ r_ik x_i z_k + Σ s_jk y_j z_k
# + Σ t_ijk x_i y_j z_k   (THIS IS THE CUBIC PART WE KEEP for DIRAC-3)
# -----------------------------
def build_hubo(df: pd.DataFrame, I_vals, J_vals, K_vals, var_idx):
    """
    Easy idea:
    - Linear terms (pick each party alone): average costs per group.
    - Pair terms (supplier with product, etc.): “savings” if lead/defect better than global average.
    - Triple terms (supplier+product+carrier): bonus/penalty for the specific trio (kept as cubic).
    """
    a = defaultdict(float); b = defaultdict(float); c = defaultdict(float)
    quad = {}   # {(i,j): coeff} with i<j
    cubic = {}  # {(i,j,k): coeff} with i<j<k

    use_car_cost = (COL_COST_CAR in df.columns)

    # Global averages = what “normal” looks like in this dataset
    g_lt   = float(df[COL_LEAD].mean())
    g_dr   = float(df[COL_DEF].mean())
    g_mc   = float(df[COL_COST_MAN].mean())
    g_cost = float(df[COL_COST_SUP].mean())
    g_car  = float(df[COL_COST_CAR].mean()) if use_car_cost else g_cost

    # Group means
    mean_cost_I = df.groupby(COL_I)[COL_COST_SUP].mean().to_dict()
    mean_mc_J   = df.groupby(COL_J)[COL_COST_MAN].mean().to_dict()
    mean_cost_K = (df.groupby(COL_K)[COL_COST_CAR].mean().to_dict()
                   if use_car_cost else df.groupby(COL_K)[COL_COST_SUP].mean().to_dict())

    # Pair means for lead time & defects
    lt_IJ = df.groupby([COL_I, COL_J])[COL_LEAD].mean()
    dr_IJ = df.groupby([COL_I, COL_J])[COL_DEF].mean()
    lt_IK = df.groupby([COL_I, COL_K])[COL_LEAD].mean()
    dr_IK = df.groupby([COL_I, COL_K])[COL_DEF].mean()
    lt_JK = df.groupby([COL_J, COL_K])[COL_LEAD].mean()
    dr_JK = df.groupby([COL_J, COL_K])[COL_DEF].mean()

    # Triple means
    lt_IJK = df.groupby([COL_I, COL_J, COL_K])[COL_LEAD].mean()
    dr_IJK = df.groupby([COL_I, COL_J, COL_K])[COL_DEF].mean()
    mc_IJK = df.groupby([COL_I, COL_J, COL_K])[COL_COST_MAN].mean()

    # Linear costs: if you pick this supplier (or product or carrier), you pay its average cost.
    for i in I_vals:
        a[var_idx[('I', i)]] = mean_cost_I.get(i, g_cost)
    for j in J_vals:
        b[var_idx[('J', j)]] = mean_mc_J.get(j, g_mc)
    for k in K_vals:
        c[var_idx[('K', k)]] = mean_cost_K.get(k, g_car)

    # Helper to accumulate symmetric quadratic keys
    def add_q(d, i, j, w):
        ii, jj = (i, j) if i < j else (j, i)
        d[(ii, jj)] = d.get((ii, jj), 0.0) + w

    # Pair “savings”: if this pair has better (lower) lead/defect than global, reward it.
    for i in I_vals:
        for j in J_vals:
            lt = float(lt_IJ.get((i, j), g_lt)); dr = float(dr_IJ.get((i, j), g_dr))
            saving = -(W_LT*(lt - g_lt) + W_DEF*(dr - g_dr))
            add_q(quad, var_idx[('I', i)], var_idx[('J', j)], saving)

    for i in I_vals:
        for k in K_vals:
            lt = float(lt_IK.get((i, k), g_lt)); dr = float(dr_IK.get((i, k), g_dr))
            saving = -(W_LT*(lt - g_lt) + W_DEF*(dr - g_dr))
            add_q(quad, var_idx[('I', i)], var_idx[('K', k)], saving)

    for j in J_vals:
        for k in K_vals:
            lt = float(lt_JK.get((j, k), g_lt)); dr = float(dr_JK.get((j, k), g_dr))
            saving = -(W_LT*(lt - g_lt) + W_DEF*(dr - g_dr))
            add_q(quad, var_idx[('J', j)], var_idx[('K', k)], saving)

    # Triple “savings”: special bonus/penalty unique to the exact trio (i,j,k).
    # We KEEP these as cubic terms; DIRAC-3 can handle them directly.
    for i in I_vals:
        for j in J_vals:
            for k in K_vals:
                lt = float(lt_IJK.get((i, j, k), g_lt))
                dr = float(dr_IJK.get((i, j, k), g_dr))
                mc = float(mc_IJK.get((i, j, k), g_mc))
                # The mix 0.8 / 0.2 / 0.5 is a design choice: how much each thing matters in triples.
                saving = -(0.8*W_LT*(lt - g_lt) + 0.2*W_DEF*(dr - g_dr) + 0.5*(mc - g_mc))
                ii = var_idx[('I', i)]; jj = var_idx[('J', j)]; kk = var_idx[('K', k)]
                aa, bb, cc = sorted([ii, jj, kk])
                cubic[(aa, bb, cc)] = cubic.get((aa, bb, cc), 0.0) + saving

    # Merge linear pieces
    lin = {}
    lin.update(a); lin.update(b); lin.update(c)
    return lin, quad, cubic

# -----------------------------
# 3) Rescale so numbers are not gigantic
# (helps solvers behave nicely)
# -----------------------------
def rescale_hubo(lin_h, quad_h, cubic_h, target_max=1e4):
    max_abs = 1.0
    if lin_h:   max_abs = max(max_abs, max(abs(v) for v in lin_h.values()))
    if quad_h:  max_abs = max(max_abs, max(abs(v) for v in quad_h.values()))
    if cubic_h: max_abs = max(max_abs, max(abs(v) for v in cubic_h.values()))
    scale = max_abs / target_max
    if scale <= 1.0:
        return lin_h, quad_h, cubic_h, 1.0
    lin_s   = {i: v/scale for i, v in lin_h.items()}
    quad_s  = {(i,j): v/scale for (i,j), v in quad_h.items()}
    cubic_s = {(i,j,k): v/scale for (i,j,k), v in cubic_h.items()}
    return lin_s, quad_s, cubic_s, scale


In [83]:
# -----------------------------
# 4) Compute penalties big enough so
# “pick exactly one” really happens
# -----------------------------
def compute_penalties(lin_h, quad_h, cubic_h, I_idx, J_idx, K_idx, safety=8.0):
    deg = defaultdict(float)
    for i, w in lin_h.items(): deg[i] += abs(w)
    for (i, j), w in quad_h.items():
        a = abs(w); deg[i] += a; deg[j] += a
    for (i, j, k), w in cubic_h.items():
        a = abs(w); deg[i] += a; deg[j] += a; deg[k] += a

    max_I = max((deg[i] for i in I_idx), default=0.0)
    max_J = max((deg[j] for j in J_idx), default=0.0)
    max_K = max((deg[k] for k in K_idx), default=0.0)

    base = max(max_I, max_J, max_K, 1.0)
    P_onehot = safety * base   # “big number” to enforce exactly one choice per group
    return P_onehot


In [84]:
# -----------------------------
# 5) Add pick-one penalties:
#   P * (sum group - 1)^2
# Expands to linear + pairwise terms only
# (still fine in a HUBO — HUBO = up to cubic; quadratic is allowed)
# -----------------------------
def add_one_hot_penalty_into_hubo(lin_h, quad_h, idx_list, P):
    # Linear: -P for each x_i
    for i in idx_list:
        lin_h[i] = lin_h.get(i, 0.0) - P
    # Quadratic: +2P x_i x_j for each distinct pair in the group
    n = len(idx_list)
    for a in range(n):
        for b in range(a+1, n):
            i, j = idx_list[a], idx_list[b]
            ii, jj = (i, j) if i < j else (j, i)
            quad_h[(ii, jj)] = quad_h.get((ii, jj), 0.0) + 2.0*P


In [85]:
# -----------------------------
# 6) Pack as a DIRAC-3 friendly polynomial dict
# (common, compact format)
# Monomial keys are tuples of variable ids, sorted, length 1/2/3
# Example: (7,) for x7; (3,9) for x3*x9; (2,4,5) for x2*x4*x5
# -----------------------------
def pack_polynomial(lin_h, quad_h, cubic_h):
    poly = {}
    for i, w in lin_h.items():
        poly[(i,)] = poly.get((i,), 0.0) + float(w)
    for (i, j), w in quad_h.items():
        ii, jj = (i, j) if i < j else (j, i)
        poly[(ii, jj)] = poly.get((ii, jj), 0.0) + float(w)
    for (i, j, k), w in cubic_h.items():
        aa, bb, cc = sorted([i, j, k])
        poly[(aa, bb, cc)] = poly.get((aa, bb, cc), 0.0) + float(w)
    return poly


In [86]:
# -----------------------------
# 7) Full builder + optional DIRAC-3 run
# -----------------------------
def build_dirac3_hubo(df: pd.DataFrame, safety=8.0, target_max=1e4, save_path="hubo_poly.json"):
    I_vals, J_vals, K_vals, var_idx, I_idx, J_idx, K_idx, base_n = build_sets(df)

    names_by_idx = {idx: f"I:{key}" for (g,key), idx in var_idx.items() if g=='I'}
    names_by_idx.update({idx: f"J:{key}" for (g,key), idx in var_idx.items() if g=='J'})
    names_by_idx.update({idx: f"K:{key}" for (g,key), idx in var_idx.items() if g=='K'})

    # 1) build native terms
    lin_h, quad_h, cubic_h = build_hubo(df, I_vals, J_vals, K_vals, var_idx)

    # (optional) light pre-scale if your raw stats are wild
    lin_h, quad_h, cubic_h, scale_used = rescale_hubo(lin_h, quad_h, cubic_h, target_max=1e4)
    print(f"[scale] pre-penalty scale 1/{scale_used:.3e}")

    # 2) compute and add penalties
    P_ONEHOT = compute_penalties(lin_h, quad_h, cubic_h, I_idx, J_idx, K_idx, safety=safety)
    print(f"[penalty] P_ONEHOT={P_ONEHOT:.2e}")

    add_one_hot_penalty_into_hubo(lin_h, quad_h, I_idx, P_ONEHOT)
    add_one_hot_penalty_into_hubo(lin_h, quad_h, J_idx, P_ONEHOT)
    add_one_hot_penalty_into_hubo(lin_h, quad_h, K_idx, P_ONEHOT)

    # (optional but helpful) gentle negative bias so “pick one” beats “pick none”
    BIAS = 0.10 * P_ONEHOT
    for i in I_idx: lin_h[i] = lin_h.get(i, 0.0) - BIAS
    for j in J_idx: lin_h[j] = lin_h.get(j, 0.0) - BIAS
    for k in K_idx: lin_h[k] = lin_h.get(k, 0.0) - BIAS

    # 3) FINAL post-penalty rescale (***this was missing***)
    lin_h, quad_h, cubic_h, scale_used2 = rescale_hubo(lin_h, quad_h, cubic_h, target_max=1e4)
    print(f"[scale] post-penalty scale 1/{scale_used2:.3e}")

    # 4) pack & write
    poly = pack_polynomial(lin_h, quad_h, cubic_h)
    out = {"nvars": max(names_by_idx) if names_by_idx else 0,
           "terms": { "-".join(map(str,k)): v for k, v in poly.items() } }
    with open(save_path, "w") as f:
        json.dump(out, f)
    print(f"[file] Wrote HUBO polynomial to {save_path}")

    return {
        "poly": poly,
        "names_by_idx": names_by_idx,
        "I_idx": I_idx, "J_idx": J_idx, "K_idx": K_idx,
        "counts": (len(I_idx), len(J_idx), len(K_idx)),
        "nvars": max(names_by_idx) if names_by_idx else 0,
        "save_path": save_path
    }


In [88]:
# === DIRAC-3 submit + decode (drop-in) ==============================
# Requires: pip install qci-client python-dotenv
load_dotenv()

API_TOKEN = os.getenv("QCI_ACCESS_TOKEN")
API_URL   = os.getenv("QCI_API_URL")

if not API_TOKEN:
    raise RuntimeError("Missing API token. Put QCI_ACCESS_TOKEN in .env")

client = qc.QciClient(api_token=API_TOKEN, url=API_URL)

# 1) load your .env (QCI_ACCESS_TOKEN / QCI_API_URL already present)
load_dotenv()
API_TOKEN = os.getenv("QCI_TOKEN") or os.getenv("QCI_ACCESS_TOKEN")
API_URL   = os.getenv("QCI_API_URL")
if not API_TOKEN:
    raise RuntimeError("Missing API token. Put QCI_ACCESS_TOKEN (or QCI_TOKEN) in .env")

# 2) read the HUBO polynomial we just wrote
with open("hubo_poly.json") as f:
    hubo = json.load(f)
nvars = int(hubo["nvars"])
terms = hubo["terms"]  # keys like "3-9-12" (1-based indices)

# 3) convert to QCi polynomial format (pad to max_degree=3; drop empty terms)
MAX_DEG = 3
EPS = 0.0  # set to e.g. 1e-12 to drop near-zero coeffs
data = []

for key, val in terms.items():                       # keys like "12", "3-9", "2-7-8"
    # guard: skip blanks/whitespace and zero-ish coefficients
    if key is None:
        continue
    key_str = str(key).strip()
    if not key_str:
        continue
    coeff = float(val)
    if abs(coeff) <= EPS:
        continue

    # parse 1-based indices; keep 1-based because 0 is the PAD sentinel in QCi
    ids = [int(t) for t in key_str.split("-") if t.strip()]

    # guard: drop zero-degree (would become [0,0,0]) and out-of-range indices
    if len(ids) == 0:
        # print(f"Skipping empty term with coeff={coeff}")
        continue
    if any(i < 1 or i > nvars for i in ids):
        # print(f"Skipping out-of-range term {ids} with coeff={coeff}")
        continue

    # left-pad with zeros so len(idx) == MAX_DEG (linear: [0,0,i], quadratic: [0,i,j], cubic: [i,j,k])
    padded = [0] * (MAX_DEG - len(ids)) + ids
    data.append({"val": coeff, "idx": padded})

# sanity checks (optional)
# assert all(len(d["idx"]) == 3 for d in data)
# assert all(any(x != 0 for x in d["idx"]) for d in data)  # no [0,0,0]
# assert len(data) > 0, "No valid terms built; check hubo_poly.json"

poly_file = {
    "file_name": "my-hubo",
    "file_config": {"polynomial": {
        "min_degree": 1,
        "max_degree": 3,
        "num_variables": nvars,   # 1-based count (since 0 is pad)
        "data": data
    }}
}


# 4) submit the job to DIRAC-3
client = qc.QciClient(api_token=API_TOKEN, url=API_URL)
file_id = client.upload_file(file=poly_file)["file_id"]

job_body = client.build_job_body(
    job_type="sample-hamiltonian",
    polynomial_file_id=file_id,
    job_params={
        "device_type": "dirac-3",
        "relaxation_schedule": 4,
        "num_samples": 40,
        "sum_constraint": 3   # hard: total ones = 3
    }
)

resp = client.process_job(job_body=job_body)
print("\n[dirac3] status:", resp.get("status"))
energies  = resp.get("results", {}).get("energies", [])
solutions = resp.get("results", {}).get("solutions", [])
if energies:  print("[dirac3] best energies:", energies[:5])
if not solutions:
    raise RuntimeError("DIRAC-3 returned no solutions.")

# 5) decode the winning combo back to names (needs your dataset + build_sets)
# If your DataFrame is called 'dataset_synthetic' (as in your code), we’ll use it.
try:
    df = dataset_synthetic  # must exist in your script
except NameError as _:
    raise RuntimeError("Please keep your DataFrame in a variable named 'dataset_synthetic'.")

# reuse your existing build_sets function
I_vals, J_vals, K_vals, var_idx, I_idx, J_idx, K_idx, base_n = build_sets(df)

# ... after you get `resp` and extract `energies`/`solutions` ...
print("\n[dirac3] status:", resp.get("status"))
energies  = resp.get("results", {}).get("energies", [])
solutions = resp.get("results", {}).get("solutions", [])
if energies:  print("[dirac3] best energies:", energies[:5])
if not solutions:
    raise RuntimeError("DIRAC-3 returned no solutions.")

# Make sure we have the dataset + index maps ready
try:
    df = dataset_synthetic  # your DataFrame variable
except NameError as _:
    raise RuntimeError("Please keep your DataFrame in a variable named 'dataset_synthetic'.")

I_vals, J_vals, K_vals, var_idx, I_idx, J_idx, K_idx, base_n = build_sets(df)
names_by_idx = {idx: f"I:{key}" for (g,key), idx in var_idx.items() if g=='I'}
names_by_idx.update({idx: f"J:{key}" for (g,key), idx in var_idx.items() if g=='J'})
names_by_idx.update({idx: f"K:{key}" for (g,key), idx in var_idx.items() if g=='K'})

# ====== REPLACE YOUR OLD DECODER WITH THIS BLOCK ======
# --- robust decoding for Dirac-3 samples (spin or soft) ---

best_vec = solutions[0]              # could be spins (-1/+1), soft [0..1], or 0/1
v = np.array(best_vec, dtype=float)  # raw vector from solver

def to_binary(vec: np.ndarray) -> np.ndarray:
    """
    Map solver outputs to {0,1} robustly:
    - If any entry < -0.5 => treat as spin {-1,+1}: b = (s + 1)/2
    - Else threshold soft values at 0.5
    """
    if np.any(vec < -0.5):            # spin format
        b = (vec + 1.0) / 2.0         # -1 -> 0, +1 -> 1
    else:
        b = np.where(vec >= 0.5, 1.0, 0.0)  # soft/real -> binary
    return b.astype(int)

# nvars came from your hubo_poly.json earlier
b = to_binary(v)
if len(b) < nvars:
    raise RuntimeError(f"Solver returned {len(b)} vars but model has {nvars}")

bits_by_id = {i+1: int(b[i]) for i in range(nvars)}  # back to 1-based ids
total_ones = int(b[:nvars].sum())
print(f"[decode] total ones in sample: {total_ones}")

def picked(group_ids):
    return [i for i in group_ids if bits_by_id.get(i, 0) == 1]

I_pick = picked(I_idx)
J_pick = picked(J_idx)
K_pick = picked(K_idx)

# fallback: if a group ended empty after thresholding, pick its strongest raw entry
def ensure_one_per_group(group_ids, picks, raw_vec):
    if picks:
        return picks
    L = len(raw_vec)
    eligible = [gid for gid in group_ids if 1 <= gid <= L]
    if eligible:
        best_i = max(eligible, key=lambda gid: raw_vec[gid-1])  # raw_vec is 0-based
    else:
        # raw_vec too short or mismatched; fallback to first group id
        best_i = group_ids[0]
    return [best_i]

I_pick = ensure_one_per_group(I_idx, I_pick, b)
J_pick = ensure_one_per_group(J_idx, J_pick, b)
K_pick = ensure_one_per_group(K_idx, K_pick, b)

def pretty(picks):
    return ", ".join(names_by_idx[p] for p in picks) if picks else "NONE"

print("\n== Optimal pick-one-of-each (DIRAC-3) ==")
print("Supplier :", pretty(I_pick))
print("Product  :", pretty(J_pick))
print("Carrier  :", pretty(K_pick))


# ===================================================================


2026-01-16 02:20:29 - Dirac allocation balance = 35751 s
2026-01-16 02:20:29 - Job submitted: job_id='69692fed08442f441bb794b2'
2026-01-16 02:20:29 - QUEUED
2026-01-16 02:20:32 - RUNNING
2026-01-16 02:21:19 - COMPLETED
2026-01-16 02:21:22 - Dirac allocation balance = 35704 s

[dirac3] status: COMPLETED
[dirac3] best energies: [-2551.7275391, -2551.6801758, -2551.626709, -2551.5449219, -2551.5441895]

[dirac3] status: COMPLETED
[dirac3] best energies: [-2551.7275391, -2551.6801758, -2551.626709, -2551.5449219, -2551.5441895]
[decode] total ones in sample: 1

== Optimal pick-one-of-each (DIRAC-3) ==
Supplier : I:Supplier 1
Product  : J:cosmetics
Carrier  : K:Carrier A


In [ ]:
pprint(resp)

{'job_info': {'job_id': '69129d898060c93397968e54',
              'job_result': {'device_usage_s': 42,
                             'file_id': '69129db5acc178773e9c0bd9'},
              'job_status': {'completed_at_rfc3339nano': '2025-11-11T02:21:41.419Z',
                             'queued_at_rfc3339nano': '2025-11-11T02:20:57.923Z',
                             'running_at_rfc3339nano': '2025-11-11T02:20:58.711Z',
                             'submitted_at_rfc3339nano': '2025-11-11T02:20:57.922Z'},
              'job_submission': {'device_config': {'dirac-3_normalized_qudit': {'num_samples': 40,
                                                                                'relaxation_schedule': 4,
                                                                                'sum_constraint': 3}},
                                 'problem_config': {'normalized_qudit_hamiltonian_optimization': {'polynomial_file_id': '69129d88acc178773e9c0bd7'}}}},
 'results': {'counts': [1,
     

Potentially Helpful to get job metrics!

In [ ]:
# --- Job metrics for your 'resp' (DIRAC-3) ---------------------------
# Requires: the same `client` you used to submit, and `resp` from process_job()
from datetime import datetime

# 1) Get job_id from resp
job_id = (resp.get("job_info", {}) or {}).get("job_id") or resp.get("job_id")
if not job_id:
    raise RuntimeError("Couldn't find job_id in resp.")

# 2) Ask the API for metrics
metrics = client.get_job_metrics(job_id=job_id)

# 3) Find the device block (e.g., 'dirac-3_qudit')
device_map = metrics["job_metrics"]["time_ns"]["device"]
device_key = next(iter(device_map.keys()))  # pick the first device present

# 4) Extract start/end timestamps (ns) and convert to seconds/timestamps
start_ns = device_map[device_key]["samples"]["start_job_ts"]
end_ns   = device_map[device_key]["samples"]["end_job_ts"]

start_ts = datetime.fromtimestamp(start_ns / 1e9)
end_ts   = datetime.fromtimestamp(end_ns / 1e9)
duration_s = (end_ns - start_ns) / 1e9

print(f"device = {device_key}")
print("device-start timestamp, device-end timestamp, duration [s]:",
      start_ts, end_ts, duration_s)

# 5) Extract each sample's runtime (ns) and convert to seconds
sample_runtimes_ns = device_map[device_key]["samples"]["runtime"]
sample_runtimes_s  = [ns / 1e9 for ns in sample_runtimes_ns]
print("sample times [s]:", sample_runtimes_s)
# ---------------------------------------------------------------------


device = dirac-3_normalized_qudit
device-start timestamp, device-end timestamp, duration [s]: 2025-11-11 10:20:58.963817 2025-11-11 10:21:41.227733 42.263916538
sample times [s]: [1.117868781, 0.884612858, 1.326123953, 0.901999831, 0.801762938, 1.198660374, 0.979905665, 0.984097898, 0.979005814, 1.331711769, 1.008920193, 0.838952303, 0.990019262, 1.090697765, 1.657951236, 1.35795486, 1.424768567, 1.043504596, 1.16141212, 1.007347584, 0.928295493, 0.909414053, 0.885145426, 1.35499227, 1.083195567, 0.971509635, 1.011087656, 0.757459402, 1.232355237, 1.197303176, 0.88821274, 1.031158328, 0.964412689, 0.863906562, 0.774702311, 0.919038951, 0.928327978, 0.883400381, 0.984655678, 0.960927129]
